# Many Models Forecasting Demo

This notebook demonstrates how to conduct fine-grained model selection after running the `mmf.run_forecast` function. Before proceeding, ensure you have run the notebooks in [`/examples/monthly`](https://github.com/databricks-industry-solutions/many-model-forecasting/tree/main/examples/monthly). You can run this notebook on a serverless compute.

In [0]:
catalog = "mmf"  # Name of the catalog we use to manage our assets
db = "m4"             # Name of the schema we use to manage our assets (e.g. datasets)

In the `scoring_output` table, forecasts for each time series from every model are stored. Let's filter by a specific time series (e.g., `M1`) and examine the forecasts from all models.

In [0]:
scoring_output =  spark.sql(f"""
    SELECT model, unique_id, date, y FROM {catalog}.{db}.monthly_scoring_output 
    WHERE unique_id='M1' ORDER BY model
    """)

display(scoring_output)

model,unique_id,date,y
ChronosBoltBase,M1,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(7868.07421875, 8148.361328125, 8273.7529296875)"
ChronosBoltMini,M1,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(7713.1787109375, 8089.353515625, 8089.353515625)"
ChronosBoltSmall,M1,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(7735.306640625, 8148.361328125, 8192.6171875)"
ChronosBoltTiny,M1,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(7576.7236328125, 8074.6015625, 8096.7294921875)"
ChronosT5Base,M1,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(8243.611328125, 8316.99609375, 8732.845703125)"
ChronosT5Large,M1,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(7876.68505859375, 7998.994140625, 8194.6875)"
ChronosT5Mini,M1,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(7411.91162109375, 7974.5322265625, 8170.22607421875)"
ChronosT5Small,M1,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(7680.99072265625, 8292.53515625, 8316.99609375)"
ChronosT5Tiny,M1,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(7680.99072265625, 8072.37890625, 8194.6875)"
MoiraiBase,M1,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(7296.91015625, 7600.63037109375, 7341.693359375)"


This table contains forecasts from 44 different models, but we need to determine which one is best for making business decisions. This is where the `evaluation_output` table becomes useful. Let's filter by a specific time series (e.g., `M1`) and review the evaluation results (i.e., backtesting trials) from all models.

In [0]:
evaluation_output =  spark.sql(f"""
    SELECT model, unique_id, backtest_window_start_date, metric_name, metric_value, forecast, actual 
    FROM {catalog}.{db}.monthly_evaluation_output where unique_id='M1'
    AND model in ('ChronosBoltBase', 'TimesFM_2_0_500m', 'MoiraiMoEBase', 'NeuralForecastAutoRNN', 'NeuralForecastAutoNHITS', 'StatsForecastBaselineNaive', 'StatsForecastAutoArima', 'SKTimeProphet')
    ORDER BY model, backtest_window_start_date
    """)

display(evaluation_output)

model,unique_id,backtest_window_start_date,metric_name,metric_value,forecast,actual
ChronosBoltBase,M1,2022-01-31T00:00:00.000Z,smape,0.05042952641046792,"List(7594.50390625, 7896.99853515625, 7844.06201171875)","List(6950.0, 7820.0, 8270.0)"
ChronosBoltBase,M1,2022-02-28T00:00:00.000Z,smape,0.05221719566183658,"List(7668.8759765625, 7713.8662109375, 7327.70458984375)","List(7820.0, 8270.0, 7840.0)"
ChronosBoltBase,M1,2022-03-31T00:00:00.000Z,smape,0.07572981533845291,"List(7875.84765625, 7452.966796875, 6907.37548828125)","List(8270.0, 7840.0, 7850.0)"
ChronosBoltBase,M1,2022-04-30T00:00:00.000Z,smape,0.060044254983780465,"List(7672.60888671875, 6998.9775390625, 6421.57958984375)","List(7840.0, 7850.0, 6710.0)"
ChronosBoltBase,M1,2022-05-31T00:00:00.000Z,smape,0.05116555096731529,"List(7012.98828125, 6475.2255859375, 6252.76953125)","List(7850.0, 6710.0, 6220.0)"
ChronosBoltBase,M1,2022-06-30T00:00:00.000Z,smape,0.07144781940611789,"List(6952.1474609375, 6687.51806640625, 6679.01708984375)","List(6710.0, 6220.0, 7430.0)"
ChronosBoltBase,M1,2022-07-31T00:00:00.000Z,smape,0.059980482722501,"List(6338.2109375, 6498.95703125, 6382.48828125)","List(6220.0, 7430.0, 6560.0)"
ChronosBoltBase,M1,2022-08-31T00:00:00.000Z,smape,0.12520473098507656,"List(6407.12939453125, 6317.31494140625, 5851.04638671875)","List(7430.0, 6560.0, 7080.0)"
ChronosBoltBase,M1,2022-09-30T00:00:00.000Z,smape,0.11749889733742391,"List(6879.9130859375, 6284.75048828125, 5543.8916015625)","List(6560.0, 7080.0, 6680.0)"
ChronosBoltBase,M1,2022-10-31T00:00:00.000Z,smape,0.16489380327946612,"List(6084.2236328125, 5551.828125, 5884.103515625)","List(7080.0, 6680.0, 6900.0)"


Based on the backtesting configuration, defined by the parameters `backtest_length`, `prediction_length`, and `stride` in the `mmf_sa.run_forecast` function, we obtain results from 10 backtesting trials for each model. For each trial, both forecasts and actual values are stored, enabling you to compute evaluation metrics based on residuals. Additionally, this table includes a built-in metric for quick assessment, which can be specified using the `metric` parameter. In this case, the metric is `smape`, and currently, `mae`, `mse`, `rmse`, `mape`, and `smape` are supported.

We compute the mean `smape` across 10 backtesting trials for each model and each time series. The model with the lowest mean `smape` is then selected for each time series, and its forecast is retrieved from the `forecast_output` table. Below is a SQL query that performs this selection.

In [0]:
forecast_best_model = spark.sql(f"""
    SELECT eval.unique_id, eval.model, eval.average_smape, score.date, score.y
    FROM 
    (
      SELECT unique_id, model, average_smape,
      RANK() OVER (PARTITION BY unique_id ORDER BY average_smape ASC) AS rank
      FROM (
        SELECT unique_id, model, AVG(metric_value) AS average_smape
        FROM {catalog}.{db}.monthly_evaluation_output
        GROUP BY unique_id, model) 
        ORDER BY unique_id, rank
    ) AS eval
    INNER JOIN {catalog}.{db}.monthly_scoring_output AS score 
      ON eval.unique_id=score.unique_id AND eval.model=score.model
    WHERE eval.rank=1
    ORDER BY eval.unique_id
    LIMIT 50
    """)

display(forecast_best_model)

unique_id,model,average_smape,date,y
M1,SKTimeProphet,0.06490791621598554,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(8506.685254588756, 8580.63398046615, 9147.60523439392)"
M10,StatsForecastBaselineSeasonalNaive,0.04339194407979583,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(12470.0, 11290.0, 13770.0)"
M100,NeuralForecastAutoRNN,0.003361166027924088,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(975.5025634765625, 977.2259521484375, 984.3120727539062)"
M1000,ChronosBoltSmall,0.11538662297873889,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(1964.634033203125, 2018.71484375, 2090.822509765625)"
M101,ChronosBoltBase,0.013454114391219879,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(1018.5897216796875, 1030.1912841796875, 1039.8592529296875)"
M102,ChronosBoltTiny,0.07223508789932025,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(3884.66455078125, 3850.48583984375, 3961.566650390625)"
M103,SKTimeProphet,0.10569316627683198,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(2980.6363576639333, 3046.3657733351747, 3228.674089325179)"
M104,StatsForecastAutoArima,0.009504299806688151,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(1511.525423699008, 1520.0508473980158, 1528.576271097024)"
M105,TimesFM_1_0_200m,0.013692023869182399,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(1182.31396484375, 1180.02099609375, 1175.67626953125)"
M106,StatsForecastAutoTbats,0.0024280608889585367,"List(2023-01-31T00:00:00.000Z, 2023-02-28T00:00:00.000Z, 2023-03-31T00:00:00.000Z)","List(1184.8633591095631, 1185.1029801401032, 1184.6267398983614)"


These forecasts will be used to guide our business decisions. Let's count how many times each model was the best-performing one.

In [0]:
model_ranking = spark.sql(f"""
    SELECT model, count(*) as count
    FROM (
      SELECT unique_id, model, average_smape,
      RANK() OVER (PARTITION BY unique_id ORDER BY average_smape ASC) AS rank
      FROM (
        SELECT unique_id, model, AVG(metric_value) AS average_smape
        FROM {catalog}.{db}.monthly_evaluation_output
        GROUP BY unique_id, model) 
        ORDER BY unique_id, rank
    ) WHERE rank=1 GROUP BY model 
    ORDER BY count DESC
    """)

display(model_ranking)

model,count
SKTimeProphet,150
TimesFM_2_0_500m,143
TimesFM_1_0_200m,92
NeuralForecastAutoRNN,53
ChronosT5Large,41
ChronosT5Base,39
StatsForecastBaselineNaive,36
StatsForecastAutoArima,31
ChronosBoltBase,25
NeuralForecastAutoNHITS,24



On this dataset (M4 monthly), it appears that TimesFM models were the best performing models based on the number of time series they excelled in.

Exposing the `evaluation_output` and `forecast_output` tables in these formats provides great flexibility in model selection. For example, you can define your own evaluation metric to compare forecasting accuracy. You can aggregate metrics using a weighted average or even the median across backtesting trials. Additionally, you can retrieve forecasts from multiple models for each time series and ensemble them. All of these options simply require writing queries against these tables.